In [76]:
import pandas as pd
import os
import nfl_data_py as nfl
from utility_db_25 import get_qb_rates, calc_tempo, count_box_bmi, get_game_pressure, mark_columns
import itertools

In [2]:
root_dir = os.getcwd()
df_games = pd.read_csv(os.path.join(root_dir,'data/games.csv'))
df_players = pd.read_csv(os.path.join(root_dir,'data/players.csv'))
df_play = pd.read_csv(os.path.join(root_dir,'data/plays.csv'))
df_player_play = pd.read_csv(os.path.join(root_dir,'data/player_play.csv'))
pfr_szn = nfl.import_seasonal_pfr('pass',[2021])
pfr_wk = nfl.import_weekly_pfr('pass',[2021])
isr = nfl.import_seasonal_rosters(years=[2021],columns=['player_id','pfr_id','gsis_it_id']).drop_duplicates()
snap_df = nfl.import_snap_counts([2021])


In [3]:
rate_df = get_qb_rates(df_play,df_player_play,df_players,df_games)

In [4]:
rate_df[['gameId','playId']].duplicated().sum()

0

In [5]:
rate_df.head(1)

,gameId,playId,qb_pa_rate_pass,qb_pass_rate_ewm
0,2022090800,56,0.275542,0.505477


In [6]:
calc_tempo(df_play)

In [7]:
list(itertools.chain(['gameId','playId','drive_pass_rate','mean_clocksnap','mean_epa','tempo'],rate_df.columns,['isDropback']))

['gameId',
 'playId',
 'drive_pass_rate',
 'mean_clocksnap',
 'mean_epa',
 'tempo',
 'gameId',
 'playId',
 'qb_pa_rate_pass',
 'qb_pass_rate_ewm',
 'isDropback']

In [8]:
play_rate = df_play.merge(rate_df,how='left')[list(itertools.chain(rate_df.columns,['drive_pass_rate','mean_clocksnap','mean_epa','tempo'],['isDropback']))]

In [9]:
play_rate.iloc[:,2:].corr()

,qb_pa_rate_pass,qb_pass_rate_ewm,drive_pass_rate,mean_clocksnap,mean_epa,tempo,isDropback
qb_pa_rate_pass,1.000000,-0.263175,-0.060639,-0.086801,0.015159,-0.072592,-0.053431
qb_pass_rate_ewm,-0.263175,1.000000,0.054171,0.076831,0.014047,0.041622,0.050620
drive_pass_rate,-0.060639,0.054171,1.000000,0.066497,0.074885,0.507578,0.090912
mean_clocksnap,-0.086801,0.076831,0.066497,1.000000,-0.001560,0.374339,0.092108
mean_epa,0.015159,0.014047,0.074885,-0.001560,1.000000,-0.725342,-0.143403
tempo,-0.072592,0.041622,0.507578,0.374339,-0.725342,1.000000,0.193662
isDropback,-0.053431,0.050620,0.090912,0.092108,-0.143403,0.193662,1.000000


In [55]:
play_rate['tempo_2'] = .1*play_rate['mean_clocksnap'] + play_rate['drive_pass_rate'] - play_rate['mean_epa']

In [11]:
play_rate.iloc[:,2:].corr()

,qb_pa_rate_pass,qb_pass_rate_ewm,drive_pass_rate,mean_clocksnap,mean_epa,tempo,isDropback,tempo_2
qb_pa_rate_pass,1.000000,-0.263175,-0.060639,-0.086801,0.015159,-0.072592,-0.053431,-0.112389
qb_pass_rate_ewm,-0.263175,1.000000,0.054171,0.076831,0.014047,0.041622,0.050620,0.167686
drive_pass_rate,-0.060639,0.054171,1.000000,0.066497,0.074885,0.507578,0.090912,0.412617
mean_clocksnap,-0.086801,0.076831,0.066497,1.000000,-0.001560,0.374339,0.092108,0.574396
mean_epa,0.015159,0.014047,0.074885,-0.001560,1.000000,-0.725342,-0.143403,-0.687578
tempo,-0.072592,0.041622,0.507578,0.374339,-0.725342,1.000000,0.193662,0.945703
isDropback,-0.053431,0.050620,0.090912,0.092108,-0.143403,0.193662,1.000000,0.194953
tempo_2,-0.112389,0.167686,0.412617,0.574396,-0.687578,0.945703,0.194953,1.000000


Joke is, essentially, that the juice of 'tempo' is mean_clocksnap, drive_pass_rate, and mean_epa. All else is marginal.

### TODO: calc_ipa

can maybe combine PA stuff, qb run stuff, with box-DL-bmi stuff? pressure? qb run val?

In [12]:
train_data=pd.read_csv(os.path.join(root_dir, "data/train_data.csv"))

In [13]:
train_data = count_box_bmi(train_data, df_players, df_player_play)
df_play = get_game_pressure(df_play,df_player_play)

In [14]:
train_data = train_data.merge(play_rate,how='left',on=['gameId','playId'])

In [15]:
train_data = train_data.merge(df_play[['gameId','playId','pressure_ewm']])

In [77]:
numeric_columns, cat_columns = mark_columns(df=train_data,features=train_data.columns, nc=None,cc = None)

In [78]:
import pandas as pd
from sklearn import preprocessing

x = train_data[numeric_columns].values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df = pd.DataFrame(x_scaled)

In [80]:
df.columns = numeric_columns

In [111]:
df['tempo_2'] = play_rate['mean_clocksnap'] + 50*play_rate['drive_pass_rate'] - play_rate['mean_epa']

In [112]:
df['isDropback'] = train_data['isDropback']

In [113]:
tc = df.corr()

In [114]:
tc['isDropback']['tempo_2']

0.025907848130670482

In [115]:
abs(tc['isDropback']).sort_values(ascending=False).head(15)

isDropback                1.000000
xpass_situational         0.488006
n_offense_backfield       0.312946
mean_pairwise_dist        0.309695
db_ct                     0.296905
weight_all_sum            0.294556
dir_standard|mean|T_1     0.244123
o_standard|mean|WR_3      0.241279
dir_standard|mean|WR_3    0.239204
dir_standard|std|WR_3     0.233680
down                      0.224330
y_standard|mean|WR_3      0.222491
n_defense_box             0.217383
x_standard|std|QB_1       0.215812
o_standard|mean|TE_2      0.214633
Name: isDropback, dtype: float64

In [105]:
tc['tempo_3'] = df['tempo_2']+

SyntaxError: invalid syntax (3512774529.py, line 1)

In [67]:
abs(tc['pressure_ewm']).sort_values(ascending=False).head(12)

prp                  1.000000
pressure_ewm         1.000000
idk                  0.986415
drive_pass_rate      0.234500
box_ewm_pre          0.197842
tempo                0.197112
box_ewm_dl_bmi       0.189285
tempo_2              0.181585
play_id              0.132684
playId               0.132684
xpass_situational    0.125661
time_remaining       0.124439
Name: pressure_ewm, dtype: float64

In [68]:
tc['box_ewm_dl_bmi']['isDropback']

-0.1286175039098842

In [69]:
tc['tempo_3']['isDropback']

0.10741174463345651

In [70]:
tc['tempo_2']['isDropback']

0.19539245011291875

In [71]:
tc['pass_val']['isDropback']

0.03228077508271429

In [72]:
def calc_ipa (data):
    ipa = (.7* data['a|max|QB_1'] + .2*data['run_val'] + \
            data['receiverAlignment_2x2'] + data['motionSinceLineset|RB_2']\
            + .1*data['s|max|G_2'] + data['surface_fieldturf'] + .05*data['time_remaining']).values
    return ipa

train_data['ipa'] = calc_ipa(train_data)
